# VS Code devcontainer and the ARC
*Jonathan Ott, Kevin Frey*

## The Goal

Have a fully reproducible code workflow for all computational analysis and/or visualisation in an **Annotated Research Context**.

> No more "But it works on my computer" and days wasted on dev environment set up.

## The solution?

Visual Studio Code [`Remote - Containers`](https://code.visualstudio.com/docs/remote/containers) extension allows the user to spawn a docker container with a set development environment.
- `devcontainer.json` with configs for the VS Code Server instance in the container.
- `Dockerfile` with the info for the dev environment.

### Requirements

- Installed Docker
- VS Code with [`Remote - Containers`](https://code.visualstudio.com/docs/remote/containers) extension

### How to start?

1. Open folder with `.devcontainer` folder on root level as workspace in VS Code.
2. Click on the new bottom left `><` button and `Reopen in container`.
3. This will create the container in docker (needs to be running) and bind the workspace and all changes into the docker dev environment.
4. All changes are computed in the docker environment but reflected to the local files.

This specific devcontainer comes with the ``.NET Interactive Notebooks`` and ``Ionide for F#`` extensions, so we can jump right into development without further requirements.

## ⚠️ .devcontainer MUST be on root level

In an *ARC* we don't want to have the .devcontainer folder in the root of the project. But instead we want to have it under *runs* or *workflows*.
So to start the devcontainer with VS Code we need to open the exact parent folder of `.devcontainer` folder. In this case `arc-bio-bte-06-l-7-small/workflows/FSharpCodeCapsule`.

This means most files in the mounted volume will not be displayed in the VS Code file explorer.

This issue is also mentioned [here](https://github.com/microsoft/vscode-remote-release/issues/2413).

### How to open/access not displayed files 

- Terminal -> New Terminal. Navgiate the whole mounted file structure with `cd` and `dir` commands until you reach the desired file. Open the file with the active VS Code instance with `code {filename}` **or** by clicking on the filename (shown by `dir`) while holding <kbd>Ctrl</kbd>.
- <kbd>Ctrl</kbd> + <kbd>O</kbd> to use VS Code `Open File...` . 

In [ ]:
// Load all nuget packages at once, so the jupyter notebook can build a dependency tree.
// If you late reference another nuget package with one of these as dependency you might get an error similiar to:
// "Cannot reference xx version x.x.x, because xx was already installed with version y.y.y"
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-preview.16"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.16"
#r "nuget: FSharp.Stats, 0.4.1"
#r "nuget: FSharpAux"
#r "nuget: BIO-BTE-06-L-7_Aux, 0.0.9"
#r "nuget: ISADotNet, 0.4.0-preview.4"
#r "nuget: ISADotNet.XLSX, 0.4.0-preview.4"
#r "nuget: ISADotNet.IO, 0.0.2"

Installed Packages BIO-BTE-06-L-7_Aux, 0.0.9 Deedle, 2.3.0 FSharp.Stats, 0.4.1 FSharpAux, 1.1.0 ISADotNet, 0.4.0-preview.4 ISADotNet.IO, 0.0.2 ISADotNet.XLSX, 0.4.0-preview.4 Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
open Deedle
open Plotly.NET
open Plotly.NET.LayoutObjects
open FSharp.Stats
open System.IO
open FSharpAux
open BIO_BTE_06_L_7_Aux.Deedle_Aux

## Review Assay file

The devcontainer has the `Office Viewer` extension installed so we can open `/workspaces/blogpost-playground/arc-bio-bte-06-l-7-small/assays/VP21_WC/isa.assay.xlsx` without leaving the VS Code environment!

Some important things to note from the isa.assay.xlsx file.

- In the `Cultivation` sheet we can see that the experiment used 3 biological replicates.
- In `Protein Extraction` we can see, that the replicates were split into 4 samples each with different quantities of Q-Protein:
    - `Parameter [15N Calvin-Benson cycle QconCAT mass#3]`
    - `Parameter [15N Photosynthesis QconCAT mass#4]`
- Any further processing was done equally on all those samples.
- All data processing ends in one file `../../assays/VP21_WC/dataset/WCAnnotated_Ratio.txt`. 
    - ⚠️ Normally the computational part should be better annotated, but for this minimal example we just have to know that the result file contains 14N/15N ratios for all peptides of the two used Q-Proteins. ⚠️

## Data analysis    

### Data access

We start by accessing our data and reading it into an [Deedle frame](https://fslab.org/packages.html).

In [ ]:
[<Literal>]
let dataPath = @"../../assays/VP21_WC/dataset/WCAnnotated_Ratio.txt"

// quality control, check dilution series. In this example median should always increase by times 5.
type Qprot =
    | CBB
    | PS

type PeptideIon = 
    {
        ProteinGroup    : string  
        Synonym         : string
        StringSequence  : string
        PepSequenceID   : int
        Charge          : int
        QProt           : Qprot
    }

let rowKeyColNames = [|"ProteinGroup"; "StringSequence"; "PepSequenceID"; "Charge"; "Synonym"|]

/// Open .xlsx file and index rows with column information. Then drop those columns
let readQuantAndProtFrame (path: string) =
    let frame =
        Frame.ReadCsv (path,true,separators="\t")
        |> Frame.indexRowsUsing (fun s ->
            let pGroup = 
                s.GetAs<string>("ProteinGroup")
                |> String.split ';'
                |> Array.sort
                |> String.concat ";"
            {
                ProteinGroup = pGroup
                StringSequence = s.GetAs<string>("StringSequence")
                PepSequenceID = s.GetAs<int>("PepSequenceID")
                Charge = s.GetAs<int>("Charge")
                Synonym = s.GetAs<string>("Synonym")
                QProt = 
                    match pGroup |> String.contains "QProt_newCBB", pGroup |> String.contains "QProt_newPS" with
                    | true, false  -> CBB
                    | false, true  -> PS  
                    | anythingElse -> failwith $"Error! unexpected input in cbb ps matching: {anythingElse}" 
            }
        )
    /// The following works on a mutable frame, so we can drop all columns we don't need without forawrding the frame to the next column to drop.
    rowKeyColNames 
    |> Array.map (fun name ->
        frame.DropColumn(name)
    )
    |> ignore
    frame

let data = readQuantAndProtFrame dataPath

In [ ]:
data 
|> Frame.take 10
|> formatAsTable 500

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
/// display table as boxplot
let createBoxPlot f =
    f
    |> Frame.getNumericCols
    |> Series.map (fun k s ->
        let x,y =
            s
            |> Series.values
            |> Seq.map (fun values -> 
                string k, values
            )
            |> Seq.unzip
        Chart.BoxPlot(x, y, Orientation = StyleParam.Orientation.Vertical)
    )
    |> Series.values
    |> Chart.combine
    |> Chart.withYAxisStyle("Ratio", MinMax=(0, 100))
    |> Chart.withLegend(false)

createBoxPlot data

<!-- Plotly chart will be drawn inside this DIV -->

This plot shows the 15N/14N ratio of all detected Q-Protein peptides.

### Metadata access

We want to access the metadata from `assays\VP21_WC\isa.assay.xlsx`. We can traverse the annotation tables of the .xlsx file with the [ISADotNet](https://github.com/nfdi4plants/ISADotNet) library.

Next we want to calculate the mean for all peptides over the biological replicates.

In [ ]:
let normalizeFileName (f: string) = if Path.HasExtension f then f else Path.ChangeExtension(f, "wiff")

open ISADotNet
open arcIO.NET

let assayFilePath = @"../../assays/VP21_WC/isa.assay.xlsx"

/// Put these functions into a separate code block. Run them once and then work in a new block, 
/// to avoid getting an error when you have the .xlsx file open somewhere else
let _,_,_,myAssayFile = XLSX.AssayFile.Assay.fromFile assayFilePath

let inOutMap = ISADotNet.createInOutMap myAssayFile

Because we know we have the sample names in the `isa.assay.xlsx` files as column headers we can use them to access related metadata information.

In [ ]:
/// Get one specific value for an output column
let getColumnParam (sheetName:string) (paramName:string) (fileName: string) =
    let fN = fileName |> normalizeFileName 
    ISADotNet.tryGetParameter // can only get Parameter [xxx]
        inOutMap 
        sheetName // Excel Sheet
        paramName // Excel column name in []
        fN // last "Sample Name" in assay file, without file extension
        myAssayFile
    |> Option.defaultValue ""
    |> String.split ' '
    |> Array.head
    |> float

/// The column parameter names in isa.assay.xlsx
let cbc_qconcat = "15N Calvin-Benson cycle QconCAT mass #3"
let ps_qconcat = "15N Photosynthesis QconCAT mass #4"

let getCBC_qconcat sampleName = getColumnParam "Protein extraction" cbc_qconcat sampleName
let getPS_qconcat sampleName = getColumnParam "Protein extraction" ps_qconcat sampleName

getColumnParam "Protein extraction" cbc_qconcat "WCGr2_U1"

0.9

### Data transformation

Map over all samples to extract Q-Protein quantities and calculate the mean over all samples with the same quantities _(the biological replicates)_.

In [ ]:
let cbc_ps_data =
    data
    /// col keys as row keys
    |> Frame.transpose
    /// Adjust keys to represent dilution series.
    |> Frame.mapRowKeys (fun rk -> sprintf "cbc: %A" (getCBC_qconcat rk), sprintf "ps: %A" (getPS_qconcat rk), rk)
    /// Calculate mean for all rows with the same value in cbc and ps q-protein quantity (aka. the biological replicates)
    |> Frame.applyLevel (fun (cbc,ps,_) -> cbc, ps) Stats.mean
    /// Swap rows keys back to column keys
    |> Frame.transpose

cbc_ps_data
|> formatAsTable 500

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:

Chart.combine [
    cbc_ps_data 
    |> createBoxPlot
    // cbc_ps_data
    // |> Frame.filterRows (fun rk s -> rk.QProt = CBB)
    // |> Frame.mapColKeys fst
    // |> createBoxPlot
    // cbc_ps_data
    // |> Frame.filterRows (fun rk s -> rk.QProt = PS)
    // |> Frame.mapColKeys snd
    // |> createBoxPlot
]

<!-- Plotly chart will be drawn inside this DIV -->

Last we want to show only ratios for peptides related to a specific protein.

In [ ]:
let plotPeptidesOf (ratios: Frame<PeptideIon,string*string>) (prot: string) = 
    ratios
    /// filter data to only show peptides of the given protein
    |> Frame.filterRows (fun k s -> k.Synonym.Contains prot || k.ProteinGroup.Contains prot)
    |> Frame.transpose
    |> Frame.getNumericCols
    |> Series.map (fun pep (values) -> 
        let qprotAmounts,ratios,fileLabel =
            values
            |> Series.map (fun dataColumnName ratio -> 
                    let cbc, ps = dataColumnName
                    let name = 
                        if pep.QProt = CBB then cbc else ps
                        |> String.split ' '
                        |> Array.last
                    name, ratio, name
            )
            |> Series.values
            |> Seq.unzip3
        Chart.Point(qprotAmounts, ratios, MultiText = fileLabel)
        |> Chart.withTraceName (sprintf "S:%s_C:%i" pep.StringSequence pep.Charge)
        |> Chart.withXAxisStyle("qProt Amount")
        |> Chart.withYAxisStyle("Ratio")
        |> Chart.withTitle (prot)
        )
    |> Series.values
    |> Chart.combine

plotPeptidesOf cbc_ps_data "rbcL"

<!-- Plotly chart will be drawn inside this DIV -->

Thats it for a minimal example on how we can interopt VS Code remote containers and the ARC.

## Verdict

- Remote containers allow scripting in an ARC to be reproducible and often with a lower set of requirements (only VS Code with the extension and docker).
- Still some issues:
    - .devcontainer must be at root level for the active VS Code workspace.
    - lacking some computational prowess (at least out of the box).
- To automatically use remote containers with CWL we would need to do nested docker containers, **plus** we just add unnecessary overhead:
    - CWL already creates a docker container for scripts to run in, with a set dev environment.
    - **Outlook** Maybe CWL can use the same Dockerfile used for the devcontainer. In this case, both would have the exact same environment,
    with remote containers for manual scripting and CWL for automated execution.